![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Card Transaction Search and Analytics with RedisVL

In this recipe, we will explore a dataset of card transactions generated by multiple users, across multiple vendors over a period of time. We will showcase the power, speed, and flexibility of the Redis Query Engine for search, filtering, vector similarity, and complex aggregations using RedisVL (Redis Vector Library).

Transaction search and analytics have many use cases - but primarily this data is useful for building realtime feature stores that can power fraud or anomaly detection machine learning models.

## What we'll cover
1. Loading transaction data into Redis
2. Vectorizing transaction data for semantic similarity search
3. Search techniques
   - Exact match filtering
   - Vector search
   - Full text search and fuzzy matching
   - Hybrid search
4. Complex aggregation queries
   - Calculate average transaction volume per week
   - Identify spending patterns
   - Generate user spending profiles

## Let's Begin!

## Prepare data

Our dataset is a list of 200 credit card transactions (fake)

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/feature-store/resources .
!rm -rf temp_repo

## Install Required Packages

In [ ]:
%pip install -q "redisvl==0.6.0" sentence-transformers pandas nltk

## Install Redis Stack

In this tutorial, Redis will be used to store, index, and query vector
embeddings created from transaction data. **We need to make sure we have a Redis
instance available**.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create Redis client and test connection

In [5]:
from redis import Redis

client = Redis.from_url(REDIS_URL)
client.ping()

True

In [6]:
# Optional: clear all data in Redis if needed
client.flushall()

True

## Load Transaction Dataset

In [7]:
import pandas as pd

# Load transactions from JSON file
df = pd.read_json("resources/transactions_200.json")
print(f"Loaded {len(df)} transaction entries")

# # Convert timestamp to datetime for easier manipulation
df["timestamp"] = df["timestamp"].apply(lambda s: int(pd.to_datetime(s).timestamp()))
df['location'] = df.apply(lambda r: f"{r.lon}, {r.lat}", axis=1)

df.head()

Loaded 200 transaction entries


,transaction_id,user_id,merchant_id,item_name,amount,currency,timestamp,lat,lon,card_provider,location
0,txn_0001,u_002,m_009,Headphones,1154.59,USD,1746182551,27.584806,-71.730465,VISA,"-71.730465, 27.584806"
1,txn_0002,u_013,m_018,Dinner,501.64,USD,1746970951,28.831898,-104.441434,AMEX,"-104.441434, 28.831898"
2,txn_0003,u_008,m_006,Laptop,1359.33,USD,1746841351,46.087128,-102.099503,VISA,"-102.099503, 46.087128"
3,txn_0004,u_011,m_024,Gaming Console,157.54,USD,1747003351,27.226349,-115.753846,VISA,"-115.753846, 27.226349"
4,txn_0005,u_010,m_014,Concert Ticket,718.00,USD,1745433751,45.108103,-79.409905,AMEX,"-79.409905, 45.108103"


### Let's examine the transaction data

In [8]:
# Basic statistics on transaction amounts
df['amount'].describe()

count     200.00000
mean      747.57135
std       426.08199
min        26.63000
25%       373.06250
50%       696.15500
75%      1130.19750
max      1499.87000
Name: amount, dtype: float64

In [9]:
# Count of transactions by card provider
df['card_provider'].value_counts()

card_provider
DISCOVER      54
AMEX          52
VISA          51
MASTERCARD    43
Name: count, dtype: int64

In [10]:
# Look at the most common items purchased
df['item_name'].value_counts().head(10)

item_name
Plane Ticket              15
Hotel Stay                15
Groceries                 14
Dinner                    14
Headphones                13
Gym Membership            13
Bicycle                   12
Gaming Console            11
Streaming Subscription    11
Smartphone                 9
Name: count, dtype: int64

In [11]:
# Look at how many users there are
df['user_id'].value_counts()

user_id
u_013    16
u_012    14
u_008    13
u_006    13
u_014    12
u_018    12
u_007    11
u_011    10
u_010    10
u_020    10
u_009     9
u_002     9
u_016     9
u_001     9
u_017     8
u_015     8
u_005     7
u_019     7
u_003     7
u_004     6
Name: count, dtype: int64

## Vectorize Transaction Data for Semantic Search

We'll use a Hugging Face sentence transformer to create vector embeddings for transaction data. The text we'll vectorize will be a combination of:
- Merchant name
- Item purchased
- Transaction amount

In [12]:
from redisvl.utils.vectorize import HFTextVectorizer

# Set environment variable to avoid parallelism warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the vectorizer with a small but powerful model
hf = HFTextVectorizer("sentence-transformers/all-MiniLM-L6-v2")

13:15:47 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
13:15:47 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.46it/s]


In [13]:
# Create a combined text field for vectorization
def create_text_for_vectorization(row):
    return f"Merchant {row['merchant_id']} selling {row['item_name']} for ${row['amount']:.2f}"

df['vector_text'] = df.apply(create_text_for_vectorization, axis=1)

# Display some examples
df[['transaction_id', 'vector_text']].head(3)

,transaction_id,vector_text
0,txn_0001,Merchant m_009 selling Headphones for $1154.59
1,txn_0002,Merchant m_018 selling Dinner for $501.64
2,txn_0003,Merchant m_006 selling Laptop for $1359.33


In [14]:
# Generate vectors for each transaction
print("Generating vectors for transactions...")
df["vector"] = hf.embed_many(df["vector_text"].tolist(), as_buffer=True)

Generating vectors for transactions...


Batches: 100%|██████████| 1/1 [00:00<00:00, 79.18it/s]


## Define Redis Index Schema

We'll create a schema that includes both standard fields and vector field for our transaction data.

In [15]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


# Define the index schema with fields we want to search and filter on
schema = IndexSchema.from_dict({
  "index": {
    "name": "transactions",
    "prefix": "transactions:entry",
    "storage_type": "hash"
  },
  "fields": [
    {
        "name": "transaction_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "user_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "merchant_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "item_name",
        "type": "text",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "amount",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "currency",
        "type": "tag",
    },
    {
        "name": "timestamp",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "card_provider",
        "type": "tag",
    },
    {
        "name": "location",
        "type": "geo",
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,                # Based on the all-MiniLM-L6-v2 model
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})

# Create the index
index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [16]:
# Check the index information
!rvl index info -i transactions -u {REDIS_URL}



Index Information:
╭────────────────────────┬────────────────────────┬────────────────────────┬────────────────────────┬────────────────────────╮
│ Index Name             │ Storage Type           │ Prefixes               │ Index Options          │ Indexing               │
├────────────────────────┼────────────────────────┼────────────────────────┼────────────────────────┼────────────────────────┤
| transactions           | HASH                   | ['transactions:entry'] | []                     | 0                      |
╰────────────────────────┴────────────────────────┴────────────────────────┴────────────────────────┴────────────────────────╯
Index Fields:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────╮
│ Name            │ Attribute       │ Type            │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Fi

## Populate Redis with Transaction Data

Now that our index is created, let's load the transaction data into Redis.

In [17]:
# Load data into Redis
transaction_ids = index.load(
    data=df.to_dict(orient="records"),
    id_field="transaction_id"
)
print(f"Loaded {len(transaction_ids)} transactions into Redis")

# Display the first few transaction IDs loaded
transaction_ids[:5]

Loaded 200 transactions into Redis


['transactions:entry:txn_0001',
 'transactions:entry:txn_0002',
 'transactions:entry:txn_0003',
 'transactions:entry:txn_0004',
 'transactions:entry:txn_0005']

# Part I: Transaction Search Techniques

Now that we have our data loaded into Redis, let's explore different search techniques.

## 1. Exact Match Queryies & Sorting

Let's start with some basic exact match filtering to find transactions with specific properties.

In [18]:
from redisvl.query import FilterQuery
from redisvl.query.filter import Tag, Num

# Find all AMEX transactions
card_filter = Tag("card_provider") == "AMEX"

query = FilterQuery(
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount", "card_provider"],
    filter_expression=card_filter,
    num_results=5
)

results = index.query(query)
pd.DataFrame(results)

,id,transaction_id,user_id,merchant_id,item_name,amount,card_provider
0,transactions:entry:txn_0002,txn_0002,u_013,m_018,Dinner,501.64,AMEX
1,transactions:entry:txn_0005,txn_0005,u_010,m_014,Concert Ticket,718,AMEX
2,transactions:entry:txn_0006,txn_0006,u_017,m_016,Hotel Stay,1232.8,AMEX
3,transactions:entry:txn_0015,txn_0015,u_001,m_018,Clothing,114.86,AMEX
4,transactions:entry:txn_0032,txn_0032,u_013,m_002,Concert Ticket,585.69,AMEX


In [19]:
# Find high-value transactions (over $1000)
amount_filter = Num("amount") > 1000

query = FilterQuery(
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount", "card_provider"],
    filter_expression=amount_filter,
    num_results=5,
).sort_by("amount", asc=False)

results = index.query(query)
pd.DataFrame(results)

,id,amount,transaction_id,user_id,merchant_id,item_name,card_provider
0,transactions:entry:txn_0061,1499.87,txn_0061,u_006,m_004,Groceries,VISA
1,transactions:entry:txn_0111,1471.73,txn_0111,u_014,m_006,Coffee,VISA
2,transactions:entry:txn_0147,1462.78,txn_0147,u_018,m_003,Dinner,MASTERCARD
3,transactions:entry:txn_0019,1462.52,txn_0019,u_012,m_005,Dinner,DISCOVER
4,transactions:entry:txn_0168,1450.52,txn_0168,u_016,m_014,Groceries,AMEX


In [20]:
# Combine filters: Find transactions for a specific user with high amounts
user_filter = Tag("user_id") == "u_013"  # Specific user
amount_filter = Num("amount") > 500      # High amount threshold

# Combine filters with logical AND
combined_filter = user_filter & amount_filter

query = FilterQuery(
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount", "timestamp"],
    filter_expression=combined_filter,
).sort_by("amount", asc=False)

results = index.query(query)
pd.DataFrame(results)

,id,amount,transaction_id,user_id,merchant_id,item_name,timestamp
0,transactions:entry:txn_0190,1413.99,txn_0190,u_013,m_005,Plane Ticket,1745570551
1,transactions:entry:txn_0145,1382.3,txn_0145,u_013,m_024,Hotel Stay,1746927751
2,transactions:entry:txn_0103,1360.17,txn_0103,u_013,m_015,Coffee,1744911751
3,transactions:entry:txn_0067,1311.19,txn_0067,u_013,m_010,Headphones,1746715351
4,transactions:entry:txn_0065,1231.32,txn_0065,u_013,m_012,Plane Ticket,1746675751
5,transactions:entry:txn_0060,1094.44,txn_0060,u_013,m_001,Ride Share,1744857751
6,transactions:entry:txn_0150,1075.13,txn_0150,u_013,m_003,Plane Ticket,1746812551
7,transactions:entry:txn_0125,1032.48,txn_0125,u_013,m_018,Shoes,1747143751
8,transactions:entry:txn_0058,916.96,txn_0058,u_013,m_024,Plane Ticket,1745523751
9,transactions:entry:txn_0113,733.8,txn_0113,u_013,m_001,Software License,1745566951


## 2. Vector Search

Now let's use vector search to find transactions semantically similar to a query.

In [21]:
from redisvl.query import VectorQuery

# Search for expensive electronics
user_query = "Expensive electronics purchase"

# Vectorize the user's query
embedded_user_query = hf.embed(user_query, as_buffer=True)

# Create vector query
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=5,
    return_fields=["transaction_id", "merchant_id", "item_name", "amount"],
    return_score=True,
).sort_by("amount", asc=False)

# Execute the query
results = index.query(vec_query)
pd.DataFrame(results)

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.19it/s]


,id,vector_distance,amount,transaction_id,merchant_id,item_name
0,transactions:entry:txn_0057,0.500209391117,1201.71,txn_0057,m_011,Laptop
1,transactions:entry:txn_0098,0.497323393822,503.01,txn_0098,m_012,Headphones
2,transactions:entry:txn_0170,0.500393152237,374.23,txn_0170,m_010,Headphones
3,transactions:entry:txn_0040,0.495004236698,159.33,txn_0040,m_017,Headphones
4,transactions:entry:txn_0169,0.494512319565,153.22,txn_0169,m_008,Headphones


## 3. Vector Search with Filters

We can combine vector search with exact match filters to get more precise results.

In [22]:
# Search for expensive purchases by a specific user
user_query = "Large purchases"
embedded_user_query = hf.embed(user_query)

# Filter for a specific user
user_filter = Tag("user_id") == "u_017"

# Create vector query with filter
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=5,
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount"],
    return_score=True,
    filter_expression=user_filter
).sort_by("amount", asc=False)

# Execute the query
results = index.query(vec_query)
pd.DataFrame(results)

Batches: 100%|██████████| 1/1 [00:00<00:00, 70.97it/s]


,id,vector_distance,amount,transaction_id,user_id,merchant_id,item_name
0,transactions:entry:txn_0042,0.533764362335,742.36,txn_0042,u_017,m_012,Groceries
1,transactions:entry:txn_0041,0.511252999306,612.59,txn_0041,u_017,m_003,Clothing
2,transactions:entry:txn_0008,0.563050031662,564.91,txn_0008,u_017,m_022,Shoes
3,transactions:entry:txn_0018,0.553927659988,462.71,txn_0018,u_017,m_013,Dinner
4,transactions:entry:txn_0195,0.5286039114,429.52,txn_0195,u_017,m_002,Groceries


In [23]:
# Search for travel expenses with price range filter
user_query = "Travel-related expenses"
embedded_user_query = hf.embed(user_query)

# Price range filter
min_amount = Num("amount") >= 500
max_amount = Num("amount") <= 1000
price_range = min_amount & max_amount

# Create vector query with filter
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=5,
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount"],
    return_score=True,
    filter_expression=price_range
)

# Execute the query
results = index.query(vec_query)
pd.DataFrame(results)

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.38it/s]


,id,vector_distance,transaction_id,user_id,merchant_id,item_name,amount
0,transactions:entry:txn_0046,0.681029856205,txn_0046,u_013,m_018,Hotel Stay,588.18
1,transactions:entry:txn_0158,0.68172955513,txn_0158,u_015,m_020,Hotel Stay,835.78
2,transactions:entry:txn_0048,0.690501689911,txn_0048,u_010,m_012,Hotel Stay,588.93
3,transactions:entry:txn_0191,0.707059979439,txn_0191,u_001,m_014,Plane Ticket,912.33
4,transactions:entry:txn_0058,0.725707709789,txn_0058,u_013,m_024,Plane Ticket,916.96


## 4. Full Text Search

Redis also provides powerful full-text search capabilities.

In [24]:
from redisvl.query import TextQuery
from redisvl.query.filter import Text

# Search for specific items
text_query = TextQuery(
    text="Gaming system, plane tickets, and hotel rooms",
    text_field_name="item_name",
    text_scorer="BM25",
    num_results=15,
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount"],
)

results = index.query(text_query)
pd.DataFrame(results)

,id,score,transaction_id,user_id,merchant_id,item_name,amount
0,transactions:entry:txn_0004,3.246753,txn_0004,u_011,m_024,Gaming Console,157.54
1,transactions:entry:txn_0013,3.246753,txn_0013,u_001,m_005,Gaming Console,293.8
2,transactions:entry:txn_0033,3.246753,txn_0033,u_008,m_021,Gaming Console,402.54
3,transactions:entry:txn_0036,3.246753,txn_0036,u_020,m_002,Gaming Console,758.65
4,transactions:entry:txn_0072,3.246753,txn_0072,u_007,m_003,Gaming Console,68.88
5,transactions:entry:txn_0088,3.246753,txn_0088,u_011,m_015,Gaming Console,26.63
6,transactions:entry:txn_0096,3.246753,txn_0096,u_014,m_021,Gaming Console,1393.99
7,transactions:entry:txn_0102,3.246753,txn_0102,u_016,m_021,Gaming Console,697.55
8,transactions:entry:txn_0109,3.246753,txn_0109,u_007,m_020,Gaming Console,43.49
9,transactions:entry:txn_0127,3.246753,txn_0127,u_001,m_021,Gaming Console,508.48


Fuzzy search is another popular technique to help with record linkage tasks.

In [25]:
from redisvl.query.filter import Text

user_filter = Tag("user_id") == "u_013"  # Specific user
fuzzy = Text("item_name") % "%%tickt%%"

fuzzy_match = FilterQuery(
    filter_expression=user_filter & fuzzy,
    return_fields=["transaction_id", "user_id", "merchant_id", "amount", "item_name"]
)

results = index.query(fuzzy_match)
pd.DataFrame(results)

,id,transaction_id,user_id,merchant_id,amount,item_name
0,transactions:entry:txn_0032,txn_0032,u_013,m_002,585.69,Concert Ticket
1,transactions:entry:txn_0058,txn_0058,u_013,m_024,916.96,Plane Ticket
2,transactions:entry:txn_0065,txn_0065,u_013,m_012,1231.32,Plane Ticket
3,transactions:entry:txn_0150,txn_0150,u_013,m_003,1075.13,Plane Ticket
4,transactions:entry:txn_0190,txn_0190,u_013,m_005,1413.99,Plane Ticket


In [26]:
# Exact text match combined with other filters
text_filter = Text("item_name") % "Gaming"  # Full text search for Laptop
card_filter = Tag("card_provider") == "VISA" # Only VISA card transactions
combined_filter = text_filter & card_filter

query = FilterQuery(
    return_fields=["transaction_id", "user_id", "merchant_id", "item_name", "amount", "card_provider"],
    filter_expression=combined_filter,
)

results = index.query(query)
pd.DataFrame(results)

,id,transaction_id,user_id,merchant_id,item_name,amount,card_provider
0,transactions:entry:txn_0004,txn_0004,u_011,m_024,Gaming Console,157.54,VISA
1,transactions:entry:txn_0036,txn_0036,u_020,m_002,Gaming Console,758.65,VISA
2,transactions:entry:txn_0102,txn_0102,u_016,m_021,Gaming Console,697.55,VISA


# Part II: Record Linkage Examples

Let's use our various search techniques to tackle a simple record linkage task. Below we have a "fake" transaction without a unique transaction ID. It may or may not be a duplicate of the data in our index already.

Because Redis is fast we can perform fast record linkage techniques and serve transaction search clients as well.

**Record linkage techniques in Redis:**
- Exact match & fuzzy text search & timestamp range
- Semantic search with vectors
- Bloom filters (probabalistic data structures -- not shown here)

In [27]:
# Create a duplicate transaction that's similar to an existing one
fake_transaction = {
    "user_id": "u_013",           # Same user as txn_0032
    "merchant_id": "m_002",       # Same merchant as txn_0032
    "item_name": "Concert Tickt", # Same item slightly mispelled
    "amount": 585.69,             # Same amount
    "currency": "USD",
    "timestamp": 1746765800,      # Very close timestamp
    "card_provider": "AMEX",      # Same card provider
    "lat": 36.173155,             
    "lon": -79.595479,            
    "location": "36.173155,-79.595479"  
}

# In this example, the transaction is a mistaken duplicate charge by the vendor

In [28]:
# User ID and Merchant ID must be the same
exact_matches = (Tag("user_id")=="u_013") & (Tag("merchant_id")=="m_002")

# Fuzzy match on Item Name
terms = fake_transaction['item_name'].split()
fuzzy_item_name = " | ".join([f"%%{term}%%" for term in terms])
fuzzy_match = Text("item_name") % fuzzy_item_name

# Timestamp range - create 60 second window on either side of transaction timestamp
from redisvl.query.filter import Timestamp

start_ts = fake_transaction['timestamp'] - 60
end_ts = fake_transaction['timestamp'] + 60
timestamp_range = Timestamp("timestamp").between(start_ts, end_ts)

# Make transaction vector
transaction_vector = hf.embed(create_text_for_vectorization(fake_transaction), as_buffer=True)

# Build query
query = VectorQuery(
    vector=transaction_vector,
    vector_field_name="vector",
    filter_expression=exact_matches & fuzzy_match & timestamp_range,
    return_fields=["user_id", "merchant_id", "item_name", "amount", "timestamp", "location"],
    num_results=3
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.15it/s]


In [29]:
str(query)

'(((@user_id:{u_013} @merchant_id:{m_002}) @item_name:(%%Concert%% | %%Tickt%%)) @timestamp:[1746765740.0 1746765860.0])=>[KNN 3 @vector $vector AS vector_distance] RETURN 7 user_id merchant_id item_name amount timestamp location vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3'

In [30]:
index.query(query)

[{'id': 'transactions:entry:txn_0032',
  'vector_distance': '0.0979611873627',
  'user_id': 'u_013',
  'merchant_id': 'm_002',
  'item_name': 'Concert Ticket',
  'amount': '585.69',
  'timestamp': '1746765751',
  'location': '-79.595479, 36.173155'}]

This kind of search op for entity resolution can be very fast!

In [31]:
%%timeit

index.query(query)

479 μs ± 19.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Part III: Complex Aggregations

Now let's explore Redis's powerful aggregation capabilities to analyze transaction data. This can be useful for feature store workloads, anomaly detection models, and even basic realtime analytics.

## 1. Basic Aggregations

First, let's look at some simple aggregations to understand spending patterns.

In [32]:
import redis.commands.search.reducers as reducers

from redisvl.redis.utils import convert_bytes, make_dict
from redisvl.query.aggregate import AggregationQuery

In [33]:
# Calculate average transaction amount by card provider
agg_query = AggregationQuery("*") \
    .group_by(
        "@card_provider",
        reducers.avg("amount").alias("avg_amount"),
        reducers.sum("amount").alias("total_amount"),
        reducers.count().alias("count")
    ) \
    .sort_by("@avg_amount")

results = index.aggregate(agg_query)
results = [make_dict(row) for row in convert_bytes(results.rows)]

pd.DataFrame(results)

,card_provider,avg_amount,total_amount,count
0,DISCOVER,661.071481481,35697.86,54
1,VISA,717.114705882,36572.85,51
2,MASTERCARD,800.729767442,34431.38,43
3,AMEX,823.311153846,42812.18,52


In [34]:
# Count transactions by item category
agg_query = AggregationQuery("*") \
    .group_by(
        "@item_name",
        reducers.sum("amount").alias("total_spent"),
        reducers.count().alias("count")
    ) \
    .sort_by("@count", max=20)


results = index.aggregate(agg_query)
results = [make_dict(row) for row in convert_bytes(results.rows)]

pd.DataFrame(results)

,item_name,total_spent,count
0,furniture,2879,3
1,camera,2700.87,5
2,laptop,4597.79,6
3,concert ticket,3771.98,6
4,software license,6070.97,8
5,shoes,7181.08,9
6,coffee,7959.15,9
7,clothing,5670.87,9
8,smartphone,6312.79,9
9,ride share,6462.65,9


## 2. User Transaction Features

Let's analyze spending profiles by user. Probably most useful for feature store workloads

In [35]:
# Let's look at some user spending features

user_filter = Tag("user_id") == "u_013"

agg_query = AggregationQuery(str(user_filter)) \
    .group_by(
        "@user_id",
        reducers.avg("amount").alias("avg_transaction_amount"),
        reducers.count().alias("transaction_count"),
        reducers.stddev("amount").alias("stdev_transaction_amount")
    )


results = index.aggregate(agg_query)
results = [make_dict(row) for row in convert_bytes(results.rows)]

pd.DataFrame(results)

,user_id,avg_transaction_amount,transaction_count,stdev_transaction_amount
0,u_013,882.1475,16,423.525319582


Let's analyze a user's recent transactions to build a feature for fraud detection.

In [36]:
# Example: Get recent transaction history for a user
user_id = "u_016"
user_filter = Tag("user_id") == user_id

# Regular search query for transactions, sorted by timestamp
query = FilterQuery(
    return_fields=["transaction_id", "timestamp", "merchant_id", "item_name", "amount", "card_provider"],
    filter_expression=user_filter,
    num_results=10
).sort_by("timestamp", asc=False)

results = index.query(query)
pd.DataFrame(results)

,id,timestamp,transaction_id,merchant_id,item_name,amount,card_provider
0,transactions:entry:txn_0121,1746312151,txn_0121,m_012,Furniture,1277.46,AMEX
1,transactions:entry:txn_0026,1746214951,txn_0026,m_016,Clothing,1166.55,DISCOVER
2,transactions:entry:txn_0102,1746168151,txn_0102,m_021,Gaming Console,697.55,VISA
3,transactions:entry:txn_0009,1745919751,txn_0009,m_022,Ride Share,259.34,DISCOVER
4,transactions:entry:txn_0198,1745797351,txn_0198,m_005,Furniture,1042.48,AMEX
5,transactions:entry:txn_0168,1745462551,txn_0168,m_014,Groceries,1450.52,AMEX
6,transactions:entry:txn_0082,1745228551,txn_0082,m_017,Streaming Subscription,1320,DISCOVER
7,transactions:entry:txn_0116,1744810951,txn_0116,m_024,Clothing,350.8,DISCOVER
8,transactions:entry:txn_0086,1744670551,txn_0086,m_005,Ride Share,528.52,VISA


In [37]:
# Calculate transaction frequency and spending patterns
user_filter = Tag("user_id") == user_id

# Using Redis aggregation functions to work with dates
agg_query = (
    AggregationQuery(str(user_filter))
    .load("@timestamp")
    .apply(ts="format('%s', @timestamp)")
    .apply(day="SUBSTR(@ts, 0, 10)")
    .group_by(
        "@day",
        reducers.count().alias("daily_transactions"),
        reducers.sum("amount").alias("daily_total")
    )
    .sort_by("@day")
)

results = index.aggregate(agg_query)
results = [make_dict(row) for row in convert_bytes(results.rows)]

pd.DataFrame(results)


,day,daily_transactions,daily_total
0,1744670551,1,528.52
1,1744810951,1,350.8
2,1745228551,1,1320
3,1745462551,1,1450.52
4,1745797351,1,1042.48
5,1745919751,1,259.34
6,1746168151,1,697.55
7,1746214951,1,1166.55
8,1746312151,1,1277.46


Let's analyze transaction patterns by geographic location.

In [38]:
# Group transactions by latitude/longitude ranges to identify geographic clusters
# For simplicity, we'll round lat/lon to the nearest whole number and group
from redisvl.query.filter import GeoRadius, Geo


geo_filter = Geo("location") == GeoRadius(-71.730465, 27.584806, 1000, "mi")

agg_query = (
    AggregationQuery(str(geo_filter))
    .group_by(
        "@merchant_id", 
        reducers.count().alias("transaction_count"),
        reducers.sum("amount").alias("transaction_totals")
    )
    .sort_by("@transaction_totals", max=20)
)

results = index.aggregate(agg_query)
results = [make_dict(row) for row in convert_bytes(results.rows)]

pd.DataFrame(results)


,merchant_id,transaction_count,transaction_totals
0,m_017,1,352.43
1,m_011,1,515.01
2,m_012,1,588.93
3,m_022,2,607.55
4,m_020,1,835.78
5,m_021,2,998.04
6,m_005,1,1000.78
7,m_019,2,1130.5
8,m_004,1,1420.3
9,m_009,3,1738.16


## Cleaning Up Redis Resources

When you're done, it's good practice to clean up your Redis resources.

In [39]:
# Clean up by deleting the index
# Uncomment the line below when you're ready to delete the index
index.delete(drop=True)
client.flushall()

True

## Conclusion

In this notebook, we've demonstrated how to:

1. Load transaction data into Redis
2. Create vector embeddings for semantic search
3. Perform various search operations:
   - Exact match filtering
   - Vector similarity search
   - Full text search
   - Using search patterns for record linkage tasks
4. Execute complex aggregation queries
   - Analyzing spending patterns by user
   - Looking at transaction volumes over time
   - Analyzing geographic transaction patterns

These capabilities make Redis and RedisVL powerful tools for building real-time feature stores that can support fraud detection, personalization, and analytics applications.